# Topic Modelling Notebook 

In [36]:
# Utilities
from joblib import Parallel, delayed
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from multiprocessing import Pool, cpu_count
import warnings 
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
tqdm.pandas(desc="progress bar")
import gc
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Import time packages 
import time
import datetime

# Visualization Tools
import matplotlib.pyplot as plt

# Deep Learning Models 
import torch
import tensorflow as tf

# Topic Modelling Packages
from bertopic import BERTopic


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [37]:
from utils import search_folder
current_dir = os.getcwd()
thesis_folder_path = current_dir.replace("thesis_code", "")
twitter_data_path = search_folder(thesis_folder_path,"twitter_data")
eikon_data_path = search_folder(thesis_folder_path,"eikon_news")
stock_data_path = search_folder(thesis_folder_path,"stock_prices")
modelling_data_path = search_folder(thesis_folder_path,"modelling_data")
google_trending_path = search_folder(thesis_folder_path,"google_search")

In [58]:
# Retrieving sentiment on cleaned text
twitter_df = pd.read_csv(twitter_data_path +"/twitter_senti_df.csv")
twitter_df["datetime"] = pd.to_datetime(twitter_df["datetime"])
twitter_df =twitter_df.sort_values(by = "datetime")
twitter_df = twitter_df.drop_duplicates()

In [59]:
# Retrieving Sentiment on Eikon Raw News 
eikon_df = pd.read_csv(eikon_data_path +"/raw_eikon_senti_df.csv")
eikon_df["datetime"] = pd.to_datetime(eikon_df["datetime"])
eikon_df =eikon_df.sort_values(by = "datetime")

In [60]:
print(twitter_df.shape)
print(eikon_df.shape)

(2157988, 21)
(44423, 16)


In [61]:
# Tweets on Saturday and Sunday are moved to Monday in order to not losse any relevance
from utils import fix_dates
twitter_df["datetime"] = twitter_df["datetime"].progress_apply(lambda x: fix_dates(x))
eikon_df["datetime"] = eikon_df["datetime"].progress_apply(lambda x: fix_dates(x))

progress bar: 100%|██████████| 44423/44423 [00:00<00:00, 481316.95it/s]


In [65]:
twitter_topics_df = twitter_df.copy()
# Convert float values to strings
twitter_topics_df["date"] = twitter_topics_df["datetime"].dt.date
twitter_topics_df["text"] = twitter_topics_df["text"].astype(str)
twitter_topics_df["cleaned_text"] = twitter_topics_df["cleaned_text"].astype(str)

# Group the rows by "date" and "company" columns and concatenate the text values
twitter_topics_df = twitter_topics_df[["date", "company", "text", "cleaned_text"]]
twitter_topics_df = twitter_topics_df.groupby(["date", "company"]).agg({
    "text": ' '.join,
    "cleaned_text": ' '.join
}).reset_index()


In [105]:
eikon_topics_df = eikon_df.copy()
# Convert float values to strings
eikon_topics_df["date"] = eikon_topics_df["datetime"].dt.date
eikon_topics_df["text"] = eikon_topics_df["text"].astype(str)
eikon_topics_df["cleaned_text"] = eikon_topics_df["cleaned_text"].astype(str)

# Group the rows by "date" and "company" columns and concatenate the text values
eikon_topics_df = eikon_topics_df[["date", "company", "text", "cleaned_text"]]
eikon_topics_df = eikon_topics_df.groupby(["date", "company"]).agg({
    "text": ' '.join,
    "cleaned_text": ' '.join
}).reset_index()


---
## Dynamic Modelling Introduction 

Dynamic topic modeling (DTM) is a collection of techniques aimed at analyzing the evolution of topics over time. These methods allow to understand how a topic is represented across different times. For example, in 1995 people may talk differently about environmental awareness than those in 2015. Although the topic itself remains the same, environmental awareness, the exact representation of that topic might differ.

In [107]:
subest_twitter_topics_df = eikon_topics_df[eikon_topics_df["company"] == "apple"]

In [108]:
timestamps = subest_twitter_topics_df.date.to_list()
timestamps = pd.to_datetime(timestamps)
tweets = subest_twitter_topics_df.cleaned_text.to_list()

Then, we need to extract the global topic representations by simply creating and training a BERTopic model:

In [120]:
from bertopic import BERTopic

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(tweets)

Batches:   0%|          | 0/51 [00:00<?, ?it/s]

2023-05-14 18:43:49,081 - BERTopic - Transformed documents to Embeddings
2023-05-14 18:43:52,611 - BERTopic - Reduced dimensionality
2023-05-14 18:43:52,650 - BERTopic - Clustered reduced embeddings


From these topics, we are going to generate the topic representations at each timestamp for each topic. We do this by simply calling topics_over_time and passing the tweets, the corresponding timestamps, and the related topics:

In [121]:
topics_over_time_eikon = topic_model.topics_over_time(tweets, timestamps, nr_bins=10)

10it [00:00, 24.98it/s]


In [97]:
topics_over_time

,Topic,Words,Frequency,Timestamp,Name
0,-1,"apple, to, the, in, for",101,2016-12-30 16:58:04.800,-1_apple_the_to_in
1,0,"apple, to, the, in, iphone",36,2016-12-30 16:58:04.800,0_apple_the_to_in
2,2,"apple, to, the, in, for",9,2016-12-30 16:58:04.800,2_apple_the_to_in
3,7,"apple, tesla, baba, spy, meta",2,2016-12-30 16:58:04.800,7_meta_tesla_cei_baba
4,8,"apple, to, the, in, of",2,2016-12-30 16:58:04.800,8_apple_the_to_and
...,...,...,...,...,...
147,14,"blk, coin, crm, cei, moderna",2,2022-08-27 16:48:00.000,14_apple_the_to_tesla
148,15,"apple, blk, meta, the, coin",2,2022-08-27 16:48:00.000,15_apple_the_to_in
149,17,"apple, the, iphone, 14, to",4,2022-08-27 16:48:00.000,17_apple_iphone_the_to
150,19,"apple, the, to, musk, and",2,2022-08-27 16:48:00.000,19_apple_the_to_and


In [111]:
topics_over_time_eikon

,Topic,Words,Frequency,Timestamp
0,-1,"apple, inc, rating, to, price",70,2016-12-30 17:08:09.600
1,0,"of, apple, update, fixing, regulator",9,2016-12-30 17:08:09.600
2,2,"dissolves, share, stake, inc, appaloosa",1,2016-12-30 17:08:09.600
3,3,"rating, apple, inc, updated, for",39,2016-12-30 17:08:09.600
4,4,"q1, assortment, sort, star, hidden",2,2016-12-30 17:08:09.600
...,...,...,...,...
112,4,"gigabit, to, apple, biden, earnings",3,2022-08-21 09:36:00.000
113,5,"the, stocks, apple, buzz, on",10,2022-08-21 09:36:00.000
114,6,"benefit, others, earnings, preview, lawsuit",1,2022-08-21 09:36:00.000
115,9,"microsoft, favourable, perseverance, platforme...",1,2022-08-21 09:36:00.000


In [96]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [122]:
topic_model.visualize_topics_over_time(topics_over_time_eikon, top_n_topics=10)

In [124]:
topic_model.visualize_barchart()

In [116]:
topic_freq = topic_model.get_topic_freq()
print(topic_freq)

    Topic  Count
0      -1    695
1       0    320
2       1    151
3       2    140
4       3     49
5       4     32
6       5     30
7       6     30
8       7     28
9       8     27
10      9     25
11     10     22
12     11     16
13     12     12
14     13     11
15     14     11
16     15     10
